In [ ]:
import pandas as pd
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt
import onekey_algo.custom.components as okcomp
from onekey_algo import get_param_in_cwd

plt.rcParams['figure.dpi'] = 300

task = get_param_in_cwd('task_column') or 'label'
bst_model = get_param_in_cwd('sel_model') or 'LR'
model_names = ['Clinic Signature', 'Rad Signature', 'Nomogram']
labels = [task]
label_data_ = pd.read_csv('clinic_sel.csv')[['ID', 'label']]
label_data_ = label_data_.dropna(axis=0)

ids = label_data_['ID']
print(label_data_.columns)
label_data = label_data_[['ID'] + labels]
label_data

# 训练集-Nomogram

In [ ]:
import pandas as pd

Clinic_results = pd.merge(pd.read_csv(f'./results/Clinic_{bst_model}_train.csv', header=0), label_data, on='ID', how='inner')
Rad_results = pd.merge(pd.read_csv(f'./results/Rad_{bst_model}_train.csv', header=0), label_data, on='ID', how='inner')

ALL_results = pd.merge(Clinic_results, Rad_results, on='ID', how='inner')
ALL_results.columns = ['ID', '-0', 'Clinic_Sig', task, '-00', 'Rad_Sig', '-l']

ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics

model = LogisticRegression(random_state=0)
# model = SVC(probability=True, random_state=0)
data_x = ALL_results[['Clinic_Sig', 'Rad_Sig']]
data_y = ALL_results[task]
model.fit(data_x, data_y)
results = model.predict_proba(data_x)
results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'{task}-0', f'{task}-1']).reset_index()
results.to_csv('./results/Nomo_train.csv', index=False, header=True)
pd.DataFrame([metrics.analysis_pred_binary(ALL_results[task], results[f'{task}-1'])], 
                  columns=['acc', 'auc', '95%CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold'])

In [ ]:
pred_column = [f'{task}-0', f'{task}-1']
Nomo_results = pd.merge(pd.read_csv('./results/Nomo_train.csv', header=0), label_data, on='ID', how='inner')
gt = [np.array(d) for d in [Clinic_results[labels], Rad_results[labels], Nomo_results[labels]]]
pred_train = [np.array(d) for d in [Clinic_results[pred_column], Rad_results[pred_column], Nomo_results[pred_column]]]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names,  
                      title=f"Model AUC")
plt.savefig(f'img/train_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
metric = []
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Train"))
pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test

Nomo_results.columns = ['ID', '-00000', 'Nomogram', '-llll']
results = pd.merge(ALL_results, Nomo_results, on='ID', how='inner')

delong = []
delong.append([delong_roc_test(results[task], results[f'Nomogram'], results[f'Clinic_Sig'])[0][0],
              delong_roc_test(results[task], results[f'Nomogram'], results[f'Rad_Sig'])[0][0], 'Train'])
pd.DataFrame(delong, columns=['Nomogram Vs Clinic', 'Nomogram Vs Rad', 'cohort'])

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([results[f'Clinic_Sig'], results[f'Rad_Sig'], results[f'Nomogram']], 
         results[task], title=f'Model for DCA', labels=model_names)
plt.savefig(f'img/train_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train, n_bins=10,
                 y_test=gt, model_names=model_names)
plt.savefig(f'img/train_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer = []
hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred[:,1], bins=15) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.DataFrame(hosmer, columns=model_names)

# 测试集-Nomogram

In [ ]:
import pandas as pd

Clinic_results = pd.merge(pd.read_csv(f'./results/Clinic_{bst_model}_test.csv', header=0), label_data, on='ID', how='inner')
Rad_results = pd.merge(pd.read_csv(f'./results/Rad_{bst_model}_test.csv', header=0), label_data, on='ID', how='inner')

ALL_results = pd.merge(Clinic_results, Rad_results, on='ID', how='inner')
ALL_results.columns = ['ID', '-0', 'Clinic_Sig', task, '-00', 'Rad_Sig', '-l']

ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics

model = LogisticRegression(random_state=0)
# model = SVC(probability=True, random_state=0)
data_x = ALL_results[['Clinic_Sig', 'Rad_Sig']]
data_y = ALL_results[task]
model.fit(data_x, data_y)
results = model.predict_proba(data_x)
results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'{task}-0', f'{task}-1']).reset_index()
results.to_csv('./results/Nomo_test.csv', index=False, header=True)
pd.DataFrame([metrics.analysis_pred_binary(ALL_results[task], results[f'{task}-1'])], 
                  columns=['acc', 'auc', '95%CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold'])

In [ ]:
pred_column = [f'{task}-0', f'{task}-1']
Nomo_results = pd.merge(pd.read_csv('./results/Nomo_test.csv', header=0), label_data, on='ID', how='inner')
gt = [np.array(d) for d in [Clinic_results[labels], Rad_results[labels], Nomo_results[labels]]]
pred_train = [np.array(d) for d in [Clinic_results[pred_column], Rad_results[pred_column], Nomo_results[pred_column]]]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names,  
                      title=f"Model AUC")
plt.savefig(f'img/test_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Test"))
metric = pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI',
                                                   'Sensitivity', 'Specificity', 
                                                   'PPV', 'NPV', 'Precision', 'Recall', 'F1',
                                                   'Threshold', 'Cohort'])

metric

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test

Nomo_results.columns = ['ID', '-00000', 'Nomogram', '-llll']
results = pd.merge(ALL_results, Nomo_results, on='ID', how='inner')

delong.append([delong_roc_test(results[task], results[f'Nomogram'], results[f'Clinic_Sig'])[0][0],
              delong_roc_test(results[task], results[f'Nomogram'], results[f'Rad_Sig'])[0][0], 'Test'])
pd.DataFrame(delong, columns=['Nomogram Vs Clinic', 'Nomogram Vs Rad', 'cohort'])

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([results[f'Clinic_Sig'], results[f'Rad_Sig'], results[f'Nomogram']], 
         results[task], title=f'Model for DCA', labels=model_names)
plt.savefig(f'img/test_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train, n_bins=5,
                 y_test=gt, model_names=model_names)
plt.savefig(f'img/test_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred[:,1], bins=5) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.DataFrame(hosmer, columns=model_names)

In [ ]:
from onekey_algo.custom.components import nomogram
import shutil

nomogram.risk_nomogram(ALL_results, result=task, columns=['Clinic_Sig', 'Rad_Sig'], width=8000, height=3000,
                      x_range='0.01,0.25,0.5,0.75,0.99')